In [1]:
from sympy import *
import numpy as np
from scipy import linalg
import sympy as sym
init_printing()

In [2]:
g = 9.81

# Mass and moment of inertia of the bus
mass = 0.5264    # The mass of the quadrotor in kg
Ixx = 0.0011990275915  # The moment of inertia of the quadrotor in
Iyy = 0.0011990275915  # The mass of the quadrotor in kg
Izz = 0.000515549122333  # The mass of the quadrotor in kg
lxy = 0.25    # The x or y distance from the quadrotor frame to the mocap markers in meters
lz = 0.046875 # The z distance from the quadrotor frame to the mocap markers in meters
Iprop = 0.00000510751666667


# drone properties

k = 1.5 * 10**-9
Larm = 0.10 #m

In [3]:
# Taken from AE 353 DeriveEOM (wes6)

# components of position (meters)
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z')

# yaw, pitch, roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')
v_in_body = sym.Matrix([v_x, v_y, v_z])

# components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')
w_in_body = sym.Matrix([w_x, w_y, w_z])

# components of net rotor torque
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')

# net rotor force
f_z = sym.symbols('f_z')

# parameters (Adding lz) (J is Intertia)
m = sym.nsimplify(mass)
Jx = sym.nsimplify(Ixx)
Jy = sym.nsimplify(Iyy)
Jz = sym.nsimplify(Izz)
l = sym.nsimplify(lxy)
g = sym.nsimplify(g)
lz = sym.nsimplify(lz)
J = sym.diag(Ixx,Iyy,Izz)

# rotation matrices (Tait-Bryant)
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])
R_body_in_world = Rz @ Ry @ Rx

# angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)

# applied forces
f_in_body = R_body_in_world.T @ sym.Matrix([[0], [0], [-m * g]]) + sym.Matrix([[0], [0], [f_z]])

# applied torques
tau_in_body = sym.Matrix([[tau_x], [tau_y], [tau_z]])

# equations of motion
f = sym.Matrix.vstack(
    R_body_in_world @ v_in_body,
    M @ w_in_body,
    (1 / m) * (f_in_body - w_in_body.cross(m * v_in_body)),
    J.inv() @ (tau_in_body - w_in_body.cross(J @ w_in_body)),
)

f = sym.simplify(f, full=True)

N(f,3)

⎡vₓ⋅cos(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)) + v_z⋅(sin(φ)⋅s
⎢                                                                             
⎢vₓ⋅sin(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)) - v_z⋅(sin(φ)⋅c
⎢                                                                             
⎢                            -vₓ⋅sin(θ) + v_y⋅sin(φ)⋅cos(θ) + v_z⋅cos(φ)⋅cos(θ
⎢                                                                             
⎢                                         w_y⋅sin(φ) + w_z⋅cos(φ)             
⎢                                         ───────────────────────             
⎢                                                  cos(θ)                     
⎢                                                                             
⎢                                         w_y⋅cos(φ) - w_z⋅sin(φ)             
⎢                                                                             
⎢                                wₓ + w_y⋅sin(φ)⋅tan

The equations of motion have this form:

$$\begin{bmatrix} \dot{p_x} \\ \dot{p_y} \\ \dot{p_z} \\  \dot{\phi} \\ \dot{\theta} \\ \dot{\psi} \\ \dot{v_x} \\ \dot{v_y} \\ \dot{v_z} \\ \dot{\omega_x} \\ \dot{\omega_y} \\ \dot{\omega_z} \end{bmatrix}=f\left(p_x, p_y, p_z, v_x, v_y, v_z, \phi, \theta, \psi, \omega_x, \omega_y, \omega_z, \tau_x, \tau_y, \tau_z, f_z\right)$$

Here is the function $f$:

In [4]:
# Linearization about Equilibrium

p_x_eq = 0
p_y_eq = 0
p_z_eq = 0
psi_eq = 0
theta_eq = 0
phi_eq = 0
v_x_eq = 0
v_y_eq = 0
v_z_eq = 0
w_x_eq = 0
w_y_eq = 0
w_z_eq = 0
tau_x_eq = 0
tau_y_eq = 0
tau_z_eq = 0
f_z_eq = m*9.81        # for force in z direction to equal 0, equilibrium established by equally counteracting gravity

# Verify Equilibrium for f
verify_feq = f.subs(psi,psi_eq).subs(theta,theta_eq).subs(phi,phi_eq).subs(v_x,v_x_eq).subs(v_y,v_y_eq).subs(v_z,v_z_eq).subs(w_x,w_x_eq).subs(w_y,w_y_eq).subs(w_z,w_z_eq).subs(tau_x,tau_x_eq).subs(tau_y,tau_y_eq).subs(tau_z,tau_z_eq).subs(f_z,f_z_eq)
if verify_feq == sym.Matrix([ [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0] ]):
    print('Equilibrium for f!')
else:
    print(verify_feq)

# Verifying Equilibrium Points

q_num = sym.lambdify([p_x,p_y,p_z,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z,tau_x,tau_y,tau_z,f_z],f)
#print(q_num(p_x_eq,p_y_eq,p_z_eq,psi_eq,theta_eq,phi_eq,v_x_eq,v_y_eq,v_z_eq,w_x_eq,w_y_eq,w_z_eq,tau_x_eq,tau_y_eq,tau_z_eq,f_z_eq))

# Calculating A (All States f)

A_num = sym.lambdify((p_x,p_y,p_z,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z,tau_x,tau_y,tau_z,f_z), f.jacobian([p_x,p_y,p_z,phi,theta,psi,v_x,v_y,v_z,w_x,w_y,w_z]))
A0 = A_num(p_x_eq,p_y_eq,p_z_eq,psi_eq,theta_eq,phi_eq,v_x_eq,v_y_eq,v_z_eq,w_x_eq,w_y_eq,w_z_eq,tau_x_eq,tau_y_eq,tau_z_eq,f_z_eq)

# Calculating B (All Inputs f)

B_num = sym.lambdify((p_x,p_y,p_z,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z,tau_x,tau_y,tau_z,f_z), f.jacobian([tau_x,tau_y,tau_z,f_z]))
B0 = B_num(p_x_eq,p_y_eq,p_z_eq,psi_eq,theta_eq,phi_eq,v_x_eq,v_y_eq,v_z_eq,w_x_eq,w_y_eq,w_z_eq,tau_x_eq,tau_y_eq,tau_z_eq,f_z_eq)


# Final Result:

A = A0
B = B0

print(A.tolist())

Equilibrium for f!
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 9.81, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, -9.81, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, -0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]


In [5]:
# Verifying Controllability and Observability

# Because Controllability implies Stability, we only test Controllability

def controllability_matrix(A, B):
    n = A.shape[0]
    controllability_mat = B
    for i in range(1, n):
        controllability_mat = np.hstack((controllability_mat, np.linalg.matrix_power(A, i) @ B))
    return controllability_mat

W = controllability_matrix(A,B)

# Check if W is square

if W.shape[0] == W.shape[1]:
    print('square')
else:
    print('W not square, must determine rank')

# Determine if W is full rank

rank_W = np.linalg.matrix_rank(W)
if rank_W == W.shape[0]:
    print('W FULL RANK, CONTROLLABLE AND ALSO STABILIZABLE')
else:
    print('W NOT FULL RANK, UNCONTROLLABLE')


W not square, must determine rank
W FULL RANK, CONTROLLABLE AND ALSO STABILIZABLE


In [6]:
# Compute K using LQR

# Establish Q Values and make Q

# MESS WITH THESE FOR STATE OUTPUT !!!!!!!!!!!!!!!!
# DO NOT CHANGE CURRENT VALUES WITHOUT TAKING A PICTURE OF THEM OR SOMETHING

Q1 = 5           # p_x , x position
Q2 = 5          # p_y , y position
Q3 = 5          # p_z , z position
Q4 = 70          # psi , yaw angle
Q5 = 70           # theta , pitch angle
Q6 = 70           # phi , roll angle
Q7 = 1.75           # v_x , linear velocity along the body-fixed x axis
Q8 = 1.75          # v_y , linear velocity along the body-fixed y axis
Q9 = 1.75         # v_z , linear velocity along the body-fixed z axis
Q10 = 0.1         # w_x , angular velocity about the body-fixed x axis (rad/s), which points forward
Q11 = 0.1       # w_y , angular velocity about the body-fixed y axis (rad/s), which points left
Q12 = 0.1          # w_z , angular velocity about the body-fixed z axis (rad/s), which points up

Q = np.diag([ Q1, Q2 , Q3 , Q4 , Q5 , Q6 , Q7 , Q8 , Q9 , Q10 , Q11 , Q12 ])

# R, for K = R_inv * B^T * P

# MESS WITH THESE FOR TORQUE OUTPUT !!!!!!!!!!!!!!!!
# DO NOT CHANGE CURRENT VALUES WITHOUT TAKING A PICTURE OF THEM OR SOMETHING
R1 = 1
R2 = 1
R3 = 1
R4 = 1

R = np.diag([ R1 , R2 , R3 , R4 ])


# ARE Equation

P = linalg.solve_continuous_are( A,B,Q,R )

# K

K = np.linalg.inv(R) @ B.T @ P

# Verifying Asymptotic Stability:

x_dot = A - B @ K

eigenvalues = np.linalg.eigvals(x_dot)
for i in eigenvalues:
    if np.real(i) < 0:
        print(f'{i} is GOOD')
    else:
        print(i)

sym.Matrix(K)

(-612.8085390210686+0j) is GOOD
(-262.39276593303896+0j) is GOOD
(-262.39271835633593+0j) is GOOD
(-26.54733044482535+0j) is GOOD
(-26.436687312659483+0j) is GOOD
(-26.593044738864645+0j) is GOOD
(-1.3835643586062658+0.8438582511076619j) is GOOD
(-1.3835643586062658-0.8438582511076619j) is GOOD
(-1.3835643247135008+0.843858291714955j) is GOOD
(-1.3835643247135008-0.843858291714955j) is GOOD
(-1.9242646574561522+0.7382784226798015j) is GOOD
(-1.9242646574561522-0.7382784226798015j) is GOOD


⎡3.06286755677357e-14  1.01477185113972e-14  1.89608859102467e-15   -2.3019338
⎢                                                                             
⎢  2.23606797749979    2.85629485722386e-14  9.24344845195912e-16  -6.03145433
⎢                                                                             
⎢4.27791752211698e-14   -2.23606797749975    4.68733572106247e-14    9.2655147
⎢                                                                             
⎣4.50152676297163e-16  1.41814455423534e-15    2.23606797749979    -5.07665114

00412e-14   -6.26607427233295e-16    8.36660026534074     2.30219054397201e-14
                                                                              
194481e-15    9.31403224085465     1.71536259343432e-16     2.44868057151606  
                                                                              
4235036     6.25131958731642e-14   -1.89789032162072e-14  5.21328071679117e-14
                                                   

Linearize the system

In [7]:
print(A.tolist())

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 9.81, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, -9.81, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, -0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]


In [8]:
print(B.tolist())

[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.8996960486322187], [834.009164667334, 0.0, 0.0, 0.0], [0.0, 834.009164667334, 0.0, 0.0], [0.0, 0.0, 1939.67937618578, 0.0]]


In [9]:
print(K.tolist())

[[3.0628675567735687e-14, 1.014771851139718e-14, 1.896088591024667e-15, -2.301933800412005e-14, -6.266074272332945e-16, 8.366600265340741, 2.3021905439720097e-14, 1.0925828606118956e-14, 8.147999239772656e-15, 0.3465019035018278, 1.6834532033275074e-16, -1.2551508128800356e-16], [2.2360679774997907, 2.8562948572238596e-14, 9.243448451959119e-16, -6.031454331944808e-15, 9.314032240854651, 1.715362593434323e-16, 2.4486805715160638, 1.5135539572907914e-14, 3.151550356657754e-16, 1.6834532033275074e-16, 0.3497650115288985, 3.4795016067452586e-16], [4.277917522116977e-14, -2.2360679774997507, 4.687335721062469e-14, 9.265514742350357, 6.251319587316424e-14, -1.8978903216207223e-14, 5.21328071679117e-14, -2.444160104367975, 1.3336675798994402e-14, -2.919140758743726e-16, 8.09237810798053e-16, 0.3309889061385077], [4.501526762971628e-16, 1.418144554235336e-15, 2.2360679774997876, -5.07665114793035e-19, -1.7477737556624022e-16, 1.3532169804252515e-15, -2.56796688536547e-17, 3.118461933289864e-1

In [10]:
P = linalg.solve_continuous_are( A,B,Q,R )

# K

K = np.linalg.inv(R) @ B.T @ P

print(K.tolist())

[[3.0628675567735687e-14, 1.014771851139718e-14, 1.896088591024667e-15, -2.301933800412005e-14, -6.266074272332945e-16, 8.366600265340741, 2.3021905439720097e-14, 1.0925828606118956e-14, 8.147999239772656e-15, 0.3465019035018278, 1.6834532033275074e-16, -1.2551508128800356e-16], [2.2360679774997907, 2.8562948572238596e-14, 9.243448451959119e-16, -6.031454331944808e-15, 9.314032240854651, 1.715362593434323e-16, 2.4486805715160638, 1.5135539572907914e-14, 3.151550356657754e-16, 1.6834532033275074e-16, 0.3497650115288985, 3.4795016067452586e-16], [4.277917522116977e-14, -2.2360679774997507, 4.687335721062469e-14, 9.265514742350357, 6.251319587316424e-14, -1.8978903216207223e-14, 5.21328071679117e-14, -2.444160104367975, 1.3336675798994402e-14, -2.919140758743726e-16, 8.09237810798053e-16, 0.3309889061385077], [4.501526762971628e-16, 1.418144554235336e-15, 2.2360679774997876, -5.07665114793035e-19, -1.7477737556624022e-16, 1.3532169804252515e-15, -2.56796688536547e-17, 3.118461933289864e-1

In [16]:
# derive A, B, and K for the yaw equilibrium

# Linearization about Equilibrium

p_x_eq = 0
p_y_eq = 0
p_z_eq = 0
psi_eq = np.pi/2
theta_eq = 0
phi_eq = 0
v_x_eq = 0
v_y_eq = 0
v_z_eq = 0
w_x_eq = 0
w_y_eq = 0
w_z_eq = 0
tau_x_eq = 0
tau_y_eq = 0
tau_z_eq = 0
f_z_eq = m*9.81        # for force in z direction to equal 0, equilibrium established by equally counteracting gravity

# Verify Equilibrium for f
verify_feq = f.subs(psi,psi_eq).subs(theta,theta_eq).subs(phi,phi_eq).subs(v_x,v_x_eq).subs(v_y,v_y_eq).subs(v_z,v_z_eq).subs(w_x,w_x_eq).subs(w_y,w_y_eq).subs(w_z,w_z_eq).subs(tau_x,tau_x_eq).subs(tau_y,tau_y_eq).subs(tau_z,tau_z_eq).subs(f_z,f_z_eq)
if verify_feq == sym.Matrix([ [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0] ]):
    print('Equilibrium for f!')
else:
    print(verify_feq)

# Verifying Equilibrium Points

q_num = sym.lambdify([p_x,p_y,p_z,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z,tau_x,tau_y,tau_z,f_z],f)
#print(q_num(p_x_eq,p_y_eq,p_z_eq,psi_eq,theta_eq,phi_eq,v_x_eq,v_y_eq,v_z_eq,w_x_eq,w_y_eq,w_z_eq,tau_x_eq,tau_y_eq,tau_z_eq,f_z_eq))

# Calculating A (All States f)

A_num = sym.lambdify((p_x,p_y,p_z,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z,tau_x,tau_y,tau_z,f_z), f.jacobian([p_x,p_y,p_z,phi,theta,psi,v_x,v_y,v_z,w_x,w_y,w_z]))
A0 = A_num(p_x_eq,p_y_eq,p_z_eq,psi_eq,theta_eq,phi_eq,v_x_eq,v_y_eq,v_z_eq,w_x_eq,w_y_eq,w_z_eq,tau_x_eq,tau_y_eq,tau_z_eq,f_z_eq)

# Calculating B (All Inputs f)

B_num = sym.lambdify((p_x,p_y,p_z,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z,tau_x,tau_y,tau_z,f_z), f.jacobian([tau_x,tau_y,tau_z,f_z]))
B0 = B_num(p_x_eq,p_y_eq,p_z_eq,psi_eq,theta_eq,phi_eq,v_x_eq,v_y_eq,v_z_eq,w_x_eq,w_y_eq,w_z_eq,tau_x_eq,tau_y_eq,tau_z_eq,f_z_eq)


# Final Result:

A2 = A0
B2 = B0

print(A2.tolist())
print(B2.tolist())

# ARE Equation

P2 = linalg.solve_continuous_are( A,B,Q,R )

# K

K2 = np.linalg.inv(R) @ B.T @ P2

print(K2.tolist())

# Verifying Asymptotic Stability:

x_dot = A - B @ K2

eigenvalues = np.linalg.eigvals(x_dot)
for i in eigenvalues:
    if np.real(i) < 0:
        print(f'{i} is GOOD')
    else:
        print(i)



# Verifying Controllability and Observability

# Because Controllability implies Stability, we only test Controllability

def controllability_matrix(A2, B2):
    n = A2.shape[0]
    controllability_mat = B2
    for i in range(1, n):
        controllability_mat = np.hstack((controllability_mat, np.linalg.matrix_power(A2, i) @ B2))
    return controllability_mat

W = controllability_matrix(A,B)

# Check if W is square

if W.shape[0] == W.shape[1]:
    print('square')
else:
    print('W not square, must determine rank')

# Determine if W is full rank

rank_W = np.linalg.matrix_rank(W)
if rank_W == W.shape[0]:
    print('W FULL RANK, CONTROLLABLE AND ALSO STABILIZABLE')
else:
    print('W NOT FULL RANK, UNCONTROLLABLE')





Equilibrium for f!
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.123233995736766e-17, -1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 6.123233995736766e-17, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 9.81, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, -9.81, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, -0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0,